<a href="https://colab.research.google.com/github/silventesa/challenge-mole/blob/francesco/transfer_learning_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning part 1

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense,GlobalAveragePooling2D,Dropout,Flatten, Conv2D, MaxPooling2D
from keras.applications.vgg16 import VGG16

# lets initialize the VGG-16 model
# We then remove the final layer of the model as we will add our own to only classify cats and dogs
# We also decide the size of the input images: here they are 64px by 64px.

prior_model = VGG16(weights='imagenet',include_top=False, input_shape=(64,64,3))

# lets create our model

model = Sequential()

# and here we add a all the VGG16 as a layer

model.add(prior_model)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/cats_dogs/"

single_prediction  test_set  training_set


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/training_set"

cats  dogs


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.layers[0].summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0     

We see that VGG-16 is composed of Conv2D layers.
and its final layer is a MaxPooling2D layer.

In order to finish our model, we need to flatten it before providing it a Dense layer for the classification.

You could add a couple of additional layers such as a Dropout or an other Dense layer before adding the softmax's one just like bellow.


In [ ]:
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 15,239,746
Trainable params: 15,239,746
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for layers in model.layers[0].layers: # looping over each layers in layer 0 to freeze them
  layers.trainable = False

model.layers[0].trainable = False

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from datetime import datetime

# compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# defining the constants for the model training

BATCH_SIZE = 32
EPOCHS = 20
URL_TRAINING = './training_set' 
URL_TESTING = './test_set' 
#google drive link
URL_TRAINING = "/content/drive/My Drive/Colab Notebooks/training_set"
URL_TESTING = "/content/drive/My Drive/Colab Notebooks/test_set"

# creating the image generator

generator = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)
test_generator = ImageDataGenerator(
    rescale=1/255,
)


# creating the train and test sets

train_set = generator.flow_from_directory(URL_TRAINING, target_size=(64,64), batch_size=BATCH_SIZE)
test_set = test_generator.flow_from_directory(URL_TESTING, target_size=(64,64), batch_size=BATCH_SIZE)

 
# fitting the model

#model.fit_generator(train_set, steps_per_epoch=len(train_set.filenames)//BATCH_SIZE, epochs=EPOCHS, validation_data = test_set, validation_steps=len(test_set.filenames)//BATCH_SIZE )
s = datetime.now()
model.fit(train_set, steps_per_epoch=len(train_set.filenames)//BATCH_SIZE, epochs=EPOCHS, validation_data = test_set, validation_steps=len(test_set.filenames)//BATCH_SIZE )
print(f'Fitting time: {str(datetime.now()-s)}')



Found 1275 images belonging to 2 classes.
Found 378 images belonging to 2 classes.
Epoch 1/20
39/39 [==============================] - 286s 7s/step - loss: 0.7533 - accuracy: 0.5968 - val_loss: 0.5587 - val_accuracy: 0.7188
Epoch 2/20
39/39 [==============================] - 5s 123ms/step - loss: 0.5346 - accuracy: 0.7183 - val_loss: 0.5828 - val_accuracy: 0.7159
Epoch 3/20
39/39 [==============================] - 5s 123ms/step - loss: 0.5108 - accuracy: 0.7375 - val_loss: 0.6274 - val_accuracy: 0.7017
Epoch 4/20
39/39 [==============================] - 5s 122ms/step - loss: 0.4818 - accuracy: 0.7752 - val_loss: 0.6468 - val_accuracy: 0.6733
Epoch 5/20
39/39 [==============================] - 5s 124ms/step - loss: 0.4951 - accuracy: 0.7590 - val_loss: 0.5624 - val_accuracy: 0.7415
Epoch 6/20
39/39 [==============================] - 5s 123ms/step - loss: 0.4887 - accuracy: 0.7624 - val_loss: 0.4771 - val_accuracy: 0.7500
Epoch 7/20
39/39 [==============================] - 5s 120ms/step 

In [ ]:
import numpy as np
from keras.preprocessing import image

TEST_IMAGE_URL = '/content/drive/My Drive/Colab Notebooks/test_image.jpg'

test_image = image.load_img( TEST_IMAGE_URL , target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(result)

[[0. 1.]]


In [ ]:
model.evaluate(test_set, steps=BATCH_SIZE)

32/32 [==============================] - 1s 23ms/step - loss: 0.5153 - accuracy: 0.7407


[0.5152507424354553, 0.7407407164573669]

In [ ]:
PATH = '/content/drive/My Drive/Colab Notebooks/64_by_64.h5'
model.save(PATH)

# Transfer learning part 2

We can see that our input layer is of shape 64,64,3.
However, since we want to increase the details of the image taken for classification, we need to increase the number of pixels the input will take in. Lets double it this time. We will now have a input shape of 128, 128, 3.
To do so, we will need to change the input layer of the VGG-16, as well as the first Conv2D layer.

Why ?
Because the first conv2D layer is mapped to the input layer, meaning that it has a shape of 64 by 64.
As we are going to increase the number of imputed pixels to 128 by 128, our new conv2D will have to be of shape 128, 128 as well, while keeping the number of filter to 64.

This means that while we froze the other layers of the VGG-16, we will have to re-train the first conv2D to learn on the new input shape, then pass forward the information to the deeper layers who will remain frozen.

Why don't we have to change the other layers as well?  Because we will max pool our new conv2D layer of shape 128,128 into a shape 64, 64 which will match the shape of the second Conv2D layer in the VGG-16 model.



Now how are we going to proceed?

We will first initiate a new model with two layers, one conv2D for input shape 128,128
then a maxpooling layer which will divide the shape of the inputs by 2 (or more precisely, the shape of the filter image by 2) as 128 / 2 = 64.


In [ ]:
model2 = Sequential()
model2.add(Conv2D(64,kernel_size=(3,3),input_shape=(128,128,3),activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

From here we just have to add the layers from the VGG-16 to our newly created model.
How so ? Since we have already frozen the VGG-16 in our previous model, we just need to loop over it and add each layers to our brand new model.

In [ ]:
for layer in model.layers[0].layers[2:]:
  # here we precise that we want to take all the layers from the second one to the last one in the VGG-16 model.
  model2.add(layer)


Finally we add all the other layers in our previous model, while freezing them in the process

In [ ]:
for layer in model.layers[1:]:
  layer.trainable = False
  model2.add(layer) 

In [ ]:
# compiling the model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# keeping the constants for the model training
# keeping the same image generator
# creating the train and test sets with different size
train_set2 = generator.flow_from_directory(URL_TRAINING, target_size=(128,128), batch_size=BATCH_SIZE)
test_set2 = test_generator.flow_from_directory(URL_TESTING, target_size=(128,128), batch_size=BATCH_SIZE)

Found 1275 images belonging to 2 classes.
Found 378 images belonging to 2 classes.


In [ ]:
# fitting the model
s = datetime.now()
model2.fit(train_set2, steps_per_epoch=len(train_set2.filenames)//BATCH_SIZE, epochs=EPOCHS, validation_data = test_set2, validation_steps=len(test_set2.filenames)//BATCH_SIZE )
print(f'Fitting time: {str(datetime.now()-s)}')

Epoch 1/20
39/39 [==============================] - 8s 212ms/step - loss: 0.6699 - accuracy: 0.6082 - val_loss: 0.6773 - val_accuracy: 0.5653
Epoch 2/20
39/39 [==============================] - 8s 208ms/step - loss: 0.6679 - accuracy: 0.6026 - val_loss: 0.6675 - val_accuracy: 0.5824
Epoch 3/20
39/39 [==============================] - 8s 208ms/step - loss: 0.6527 - accuracy: 0.6090 - val_loss: 0.6713 - val_accuracy: 0.5710
Epoch 4/20
39/39 [==============================] - 8s 211ms/step - loss: 0.6519 - accuracy: 0.6163 - val_loss: 0.6464 - val_accuracy: 0.6193
Epoch 5/20
39/39 [==============================] - 8s 211ms/step - loss: 0.6506 - accuracy: 0.6283 - val_loss: 0.6567 - val_accuracy: 0.5909
Epoch 6/20
39/39 [==============================] - 8s 209ms/step - loss: 0.6600 - accuracy: 0.6098 - val_loss: 0.6621 - val_accuracy: 0.5824
Epoch 7/20
39/39 [==============================] - 8s 209ms/step - loss: 0.6537 - accuracy: 0.6090 - val_loss: 0.6670 - val_accuracy: 0.5881
Epoch 

In [ ]:
model2.evaluate(test_set2, steps=BATCH_SIZE)

32/32 [==============================] - 1s 26ms/step - loss: 0.6391 - accuracy: 0.6085


[0.6391320824623108, 0.6084656119346619]

In [ ]:
TEST_IMAGE_URL = '/content/drive/My Drive/Colab Notebooks/test_image.jpg'

test_image = image.load_img( TEST_IMAGE_URL , target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result2 = model2.predict(test_image)
print(result2)

[[6.2735995e-21 1.0000000e+00]]


In [ ]:
TEST_IMAGE_URL2 = '/content/drive/My Drive/Colab Notebooks/test_image2.jpg'

In [ ]:
test_image2 = image.load_img( TEST_IMAGE_URL2 , target_size = (128, 128))
test_image2 = image.img_to_array(test_image2)
test_image2 = np.expand_dims(test_image2, axis = 0)
result22 = model2.predict(test_image2)
print(result22)

[[6.2157044e-09 1.0000000e+00]]
